<a href="https://colab.research.google.com/github/chandu389/trading/blob/main/momentum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import yfinance as yf
import pandas as pd
from datetime import timedelta, date
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Read ticker data from a local file or Google Drive
tickers = pd.read_csv('/content/drive/My Drive/Trading/ind_niftytotalmarket_list.csv')
tickers['NSSymbol'] = tickers['Symbol'] + ".NS"

In [4]:
start_date = date.today() - timedelta(weeks=52)
end_date = date.today() - timedelta(weeks=4)

In [8]:
print(start_date, end_date, date.today())

2023-06-19 2024-05-20 2024-06-17


In [9]:
stock_data = []
for ticker in tickers['NSSymbol']:
    raw_data = yf.download(ticker, start=start_date, end=end_date, interval='1d')
    raw_data['Symbol'] = ticker
    stock_data.append(raw_data)
stock_data = pd.concat(stock_data)
stock_data.reset_index(inplace=True)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [10]:
stock_data

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,2023-06-19,441.149994,446.000000,439.700012,441.200012,429.518219,108424.0,360ONE.NS
1,2023-06-20,438.600006,443.450012,433.850006,441.200012,429.518219,965581.0,360ONE.NS
2,2023-06-21,441.700012,449.399994,438.649994,439.950012,428.301270,512002.0,360ONE.NS
3,2023-06-22,443.149994,462.500000,441.149994,452.450012,440.470306,600719.0,360ONE.NS
4,2023-06-23,454.899994,470.500000,419.850006,450.600006,438.669312,491203.0,360ONE.NS
...,...,...,...,...,...,...,...,...
164344,2024-05-13,740.599976,756.900024,732.000000,752.299988,752.299988,150953.0,EMUDHRA.NS
164345,2024-05-14,755.950012,764.349976,751.349976,760.299988,760.299988,93390.0,EMUDHRA.NS
164346,2024-05-15,763.700012,778.400024,745.000000,753.599976,753.599976,114109.0,EMUDHRA.NS
164347,2024-05-16,759.849976,798.400024,759.650024,793.700012,793.700012,287536.0,EMUDHRA.NS


In [11]:
stock_data['WeekStartDate'] = stock_data['Date'] - pd.to_timedelta(stock_data['Date'].dt.dayofweek, unit='D')

In [17]:
stock_data['WeekDayNumber'] = stock_data['Date'].dt.weekday

In [18]:
stock_data

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,WeekStartDate,WeekDayNumber
0,2023-06-19,441.149994,446.000000,439.700012,441.200012,429.518219,108424.0,360ONE.NS,2023-06-19,0
1,2023-06-20,438.600006,443.450012,433.850006,441.200012,429.518219,965581.0,360ONE.NS,2023-06-19,1
2,2023-06-21,441.700012,449.399994,438.649994,439.950012,428.301270,512002.0,360ONE.NS,2023-06-19,2
3,2023-06-22,443.149994,462.500000,441.149994,452.450012,440.470306,600719.0,360ONE.NS,2023-06-19,3
4,2023-06-23,454.899994,470.500000,419.850006,450.600006,438.669312,491203.0,360ONE.NS,2023-06-19,4
...,...,...,...,...,...,...,...,...,...,...
164344,2024-05-13,740.599976,756.900024,732.000000,752.299988,752.299988,150953.0,EMUDHRA.NS,2024-05-13,0
164345,2024-05-14,755.950012,764.349976,751.349976,760.299988,760.299988,93390.0,EMUDHRA.NS,2024-05-13,1
164346,2024-05-15,763.700012,778.400024,745.000000,753.599976,753.599976,114109.0,EMUDHRA.NS,2024-05-13,2
164347,2024-05-16,759.849976,798.400024,759.650024,793.700012,793.700012,287536.0,EMUDHRA.NS,2024-05-13,3


In [ ]:
# Group by 'Date' and 'WeekStartDate', and select the maximum 'WeekDayNumber' within each group
grouped_df = stock_data.groupby(['Symbol', 'WeekStartDate'])['WeekDayNumber'].max().reset_index()
# Merge to get the close price for max weekday number
friday_close_data = pd.merge(stock_data, grouped_df, on=['Symbol', 'WeekStartDate', 'WeekDayNumber'], how='inner')

In [ ]:
# Calculate the stock price four weeks ago for each date
friday_close_data['FourWeeksAgo'] = friday_close_data['WeekStartDate'] - pd.DateOffset(weeks=4)

# Perform a join operation to get the stock price four weeks ago
momentum_data = friday_close_data.merge(friday_close_data[['WeekStartDate', 'Symbol', 'Close']],
                                        left_on=['Symbol', 'FourWeeksAgo'], right_on=['Symbol', 'WeekStartDate'],
                                        suffixes=('', '_FourWeeksAgo'), how='left')
# Calculate the momentum for every week
momentum_data['Momentum'] = 1 + ((momentum_data['Close'] - momentum_data['Close_FourWeeksAgo']) /
                                 momentum_data['Close_FourWeeksAgo'])
mom_by_ticker = momentum_data.groupby(['Symbol'])['Momentum'].agg('prod') - 1
mom_by_ticker = mom_by_ticker.reset_index()